# Symbolic calculation for covariant transform on the SSR group
### [Vladimir V Kisil](http://v-v-kisil.scienceontheweb.net/)

This is a [working Jupyter notebook](https://github.com/vvkisil/SSR-group-computations) which was used for research published in [Ref.4](#refKisil21c). A readable narrative for this notebook is recorded in [Ref.7](#refAlqurashiKisil23a). 

The notebook is very detailed with numerous cross-checks of the correctness and supplementary computation. It may serve also as a demonstration of [pyGiNaC](#http://moebinv.sourceforge.net/pyGiNaC.html) abilities for symbolic computations in actual research environment. 

**Note:** *to execute the notebook you may need to [install software](https://colab.research.google.com/github/vvkisil/MoebInv-notebooks/blob/master/Introduction/Software_installation_GUI_integration.ipynb).* If you do not have a local installation of  [pyGiNaC](#http://moebinv.sourceforge.net/pyGiNaC.html) then you can use a CodeOcean capsule [Ref.5](#refKisil21b).

There is also the [processed HTML](http://v-v-kisil.scienceontheweb.net/SSR-group-computations/ssr_group.html).

<a id="ToC"></a>
#### Table of contents of this notebook

+ [Initialisation, auxiliary functions and variables](#initialisation)

+ [SSR group](#ssrGroup)
  - [Lie algebra of the SSR group](#LieAlgebra)
  - [Exponential map](#exponentialMap)
    * [Elements conversion](#elementsConversion)
  - [Lie group definition](#LieGroup)
  
+ [Subgroups and homogeneous spaces](#subgroupsHS)
  - [Maximal abelian subgroup](#maxabelianSubgroup)
  - [The centre of the SSR group](#centreSSRGroup)

+ [Induced representations of the SSR group](#indeycedRepresentations)
  - [Definition of the symbolic functions](#definitionFunctions)
  - [Representations of the SSR group induced from the centre](#inductionCentre)
    * [Infinitesimal (derived) representations](#infinitesimalCentre)
  - [The representation induced from the maximal abelian subgroup](#inducedMaximal)  
+ [Coherent states](#coherentStates)
  - [Identifying the vacuum vectors](#identifyingVacuum)
  - [Case of non-zero derivative in R direction (Er ≠ 0)](#transmutationErNonZero)
  - [The case of vanishing derivative in R direction (Er = 0)](#transmutationErZero)
  - [Annihilating operators of Cauchy–Riemann type (first order)](#CRannihilator)
    * [The phase-space pair](#CRphaseSpace)
    * [Affine group analyticity](#CRaffineGroup)
    * [Complex symbolic variables](#complexVariables)
  - [Wave/Schrödinger annihilators (second order)](#secondOrderAnnihilator)
    * [Second order annihilator for B](#secondOrderAnnihilatorB)
    * [Second order annihilator for R](#secondOrderAnnihilatorR)
    * [An example: wave packet](#exampleWavePacket)
    * [An example: plane wave](#examplePlaneWave)
+ [Metamorphism](#metamorphism)
  - [Basic properties of the metamorphism](#metamorphismBasics)
  - [Solving the image of the Helmholtz equation](#solvingHelmholtz)
    * [Example: plane wave solution](#solvingPlaneWave)
    * [Example: wave packet solution](#solvingWavePaclet)
+ [Generic solution of the Helmholtz equation](#HelmholtzGeneric)
  - [Cartesian coordinates representation](#CartesianRepresentation)
  - [Plane wave from the partial solution](#partialPlaneWave)
  - [Jacobi (Hankel) functions](#JacobiFunctions)
  - [Fundamental solution](#fundamentalSolution)
+ [Helmholtz 3D](#Helmholtz3D)
  - [Generic solution of the Helmholtz equation in 3D](#generalHelmholtz3D)
+ [References](#references)


<a id="initialisation"></a>
## Initialisation, auxiliary functions and variables 

Initialise the system by loading `pyGiNaC`.

In [ ]:
from ginac import *
from IPython.display import Latex
#latex_off()
latex_on()

Fine-tine $\LaTeX$ output of calculations.

In [ ]:
def bool_highlighted(ginac):
    """ Print in bold fase boolean value """
    if is_latex_on():
        ginac = ginac.replace("True", "\\textbf{True}")
        ginac = ginac.replace("False", "\\textbf{False}")
        return Latex(ginac)
    else:
        ginac = ginac.replace("True", "_True_")
        ginac = ginac.replace("False", "_False_")
        return str(ginac)

def print_plain(String, Expr):
    """ One-off output without LaTeX formatting """
    was_latex = is_latex_on()
    latex_off()
    print(String, Expr)
    if (was_latex):
        latex_on()

We will need to use the following algebraic transformations regularly to simplify expressions

In [ ]:
power_laws = {power(wild(9),wild(10))*wild(11)*power(wild(9),wild(12))\
              : power(wild(9), (wild(10)+wild(12)))*wild(11), \
    power(wild(1),wild(2))*power(wild(1),wild(3)) : power(wild(1),(wild(2)+wild(3))),\
    power(power(wild(0),wild(1)),wild(2)) : power(wild(0),wild(1)*wild(2))}
extra_power_laws = {power(wild(8),wild(10))*wild(11)*power(wild(9),wild(10))\
                    : power(wild(8)*wild(9), wild(10))*wild(11),\
                    power(wild(1),wild(2))*power(wild(3),wild(2))\
                    : power(wild(1)*wild(3), wild(2))}
exponent_laws = {power(exp(wild(5)), wild(6))  :  exp(wild(5)*wild(6)), \
                 exp(wild(0))*exp(wild(1)) : exp(wild(0)+wild(1)), \
                 exp(wild(2))*wild(3)*exp(wild(4)) : exp(wild(2)+wild(4))*wild(3)}

def apply_laws(Expr, with_powers = False, options = 0):
    """
    Apply advanced transformation eules to the expression Expr.
    By default it includes exponent laws.
    If with_powers set to True, then additionally power laws are applied as well.
    Additional options to GiNaC subs method can be provided also.
    """
    X = Expr.normal()\
        .subs({pow(exp(wild(5)),-1) : exp(-wild(5))})\
        .subs(exponent_laws, options).normal()\
        .expand().subs(exponent_laws, options).normal()
    if with_powers:
        return X.expand().subs(power_laws, options).normal()
    else:
        return X

Some mnemonic values for logical parameters.

In [ ]:
with_power_laws = True
Left_action = True
Right_action = False

[Back to ToC](#ToC)

<a id="ssrGroup"></a>
## SSR group

First, we construct the machinery to work with SSR group. The process is as follows:

1. We realise the [Lie algebra](#LieAlgebra) through matrices to represent known commutators.
2. The [exponential map](#exponentialMap) is used to find the group law
3. The obtained expressions are implemented into procedures for speed.

<a id="LieAlgebra"></a>
### Lie algebra of the SSR group

Some initial setting: names of vector fields in the Lie algebra basis.

In [ ]:
fields = ['S', 'X', 'Y', 'B', 'R']

We start from the matrix representation of the Lie algebra.

In [ ]:
B0=symbolic_matrix(4,4,"b")
X=matrix([[0,0,1,0],[0,0,0,1],[0,0,0,0],[0,0,0,0]])
Y=matrix([[0,-1,0,0],[0,0,0,0],[0,0,0,1],[0,0,0,0]])
S=matrix([[0,0,0,2],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
R=matrix([[0,0,0,0],[0,1,0,0],[0,0,-1,0],[0,0,0,0]])
B=matrix([[0,0,0,0],[0,0,-1,0],[0,0,0,0],[0,0,0,0]])

*Excercise*. We check that the commutators of the representing matrices are right.

In [ ]:
display(bool_highlighted("[X,Y]=S: "+f'${(X*Y-Y*X-S).evalm().is_zero_matrix()}$'))
display(bool_highlighted("[R,X]=X: "+f'${(R*X-X*R-X).evalm().is_zero_matrix()}$'))
display(bool_highlighted("[R,Y]=-Y: "+f'${(R*Y-Y*R+Y).evalm().is_zero_matrix()}$'))
display(bool_highlighted("[R,S]=0: "+f'${(R*S-S*R).evalm().is_zero_matrix()}$'))
display(bool_highlighted("[B,X]=0: "+f'${(B*X-X*B).evalm().is_zero_matrix()}$'))
display(bool_highlighted("[B,Y]=-X: "+f'${(B*Y-Y*B+X).evalm().is_zero_matrix()}$'))
(bool_highlighted("[R,B]=2B: "+f'${(R*B-B*R-2*B).evalm().is_zero_matrix()}$'))

*Excercise*. We also check nilpotency of the basis elements, \(R\) shall be the
only exception.

In [ ]:
vectors={"S": S, "X": X, "Y" : Y, "R" : R, "B" : B}
for v in vectors:
    display(bool_highlighted(f'{v} is nilpotent: ${pow(vectors[v],2).evalm().is_zero_matrix()}$'))

[Back to ToC](#ToC)

<a id="exponentialMap"></a>
### Exponential map

The SSR group is constructed through the exponential map of the above Lie
algebra.  Here is a matrix for a generic element.

In [ ]:
s=realsymbol("s")
x=realsymbol("x")
y=realsymbol("y")
r=possymbol("r")
b=realsymbol("b")
u=realsymbol("u")
u2=realsymbol("u2")
I4=unit_matrix(4)
expR=matrix([[1,0,0,0],[0,r,0,0],[0,0,1/r,0],[0,0,0,1]])
g=((I4+s*S)*(I4+y*Y)*(I4+x*X)*(I4+b*B)*expR).evalm()
Latex(f'Generic SSR group element: ${g}$')

More generally,we represent a generic element of the Schrödinger–Jacobi group, the group SSR is its subgroup, cf. [Ref.6](#EichlerZagier85), Thm.6.1.

In [ ]:
a=realsymbol("a")
b=realsymbol("b")
c=realsymbol("c")
d=realsymbol("d")

Sch=((I4+s*S)*(I4+y*Y)*(I4+x*X)*matrix([[1,0,0,0],[0,a,b,0],[0,c,d,0],[0,0,0,1]])).evalm()
Latex(f'Generic Schrödinger–Jacobi group element: ${Sch}$')

[Back to ToC](#ToC)

<a id="elementsConversion"></a>
#### Elements conversion

We define several convenience routines, which convert between different forms of the SSR group.

The next routine converts a tuple into the matrix representing of the respective SSR element. 

In [ ]:
def tuple_to_ssr_matrix(s, x, y, b, r=1):
    """Convert a 4/5-tuple into a matrix representing SSR group""" 
    return matrix([[1,-y*r, (x+b*y)/r, 2*s-x*y],\
                    [ 0, r, -b/r, x],\
                    [ 0, 0, 1/r, y],\
                    [ 0, 0, 0, 1]])

Alternatively we make an SSR element as a vector from a list or elements

In [ ]:
def ssr_element(*args):
    """ Create an element of the SSR group a vector from a list or elements. """
    if len(args) == 1:
        if type(args) is list:
            return matrix([].append(args))
        elif type(args) is cginac.matrix:
            return args
    elif len(args) == 4:
        return matrix([[args[0], args[1], args[2], args[3], 1]])
    elif len(args) == 5:
        return matrix([[args[0], args[1], args[2], args[3], args[4]]])
    else:
        print("Do not recognise an argument type")

Also we make SSR matrix from a vector or a list.

In [ ]:
def to_ssr_matrix(L):
    """ Make SSR matrix from a vector or a list."""
    if L.nops() == 4:
        return tuple_to_ssr_matrix(L.op(0), L.op(1), L.op(2), L.op(3), 1)
    else:
        return tuple_to_ssr_matrix(L.op(0), L.op(1), L.op(2), L.op(3), L.op(4))

The converse operation: from matrix to the group element.

In [ ]:
def from_ssr_matrix(M1):
    """Convert a suitable 4x4 matrix into 5-tuple for the SSR matrix""" 
    M=evalm(M1)
    if (not ((M.op(1) + M.op(2*4+3)*M.op(4+1)).normal().is_zero() and \
        (M.op(2)*M.op(4+1)-M.op(4+3)-M.op(1)*M.op(4+2)).normal().is_zero())):
        print("Matrix is of the wrong form")
        return [0, 0, 0, 0]
    else:
        return matrix([[((M.op(3)+M.op(4+3)*M.op(2*4+3))/2).normal(), M.op(4+3), M.op(2*4+3), -M.op(4+2)*M.op(4+1), M.op(4+1)]])

Define some generic elements of SSR group for the symbolic computations.

In [ ]:
G0=matrix([[s,x,y,b,1]])
G=matrix([[s,x,y,b,r]])

s1=realsymbol("s'")
x1=realsymbol("x'")
y1=realsymbol("y'")
b1=realsymbol("b'")
r1=possymbol("r'")

G1=matrix([[s1,x1,y1,b1,r1]])
G10=matrix([[s1,x1,y1,b1,1]])

s2=realsymbol("s''")
x2=realsymbol("x''")
y2=realsymbol("y''")
b2=realsymbol("b''")
r2=possymbol("r''")

G2=matrix([[s2,x2,y2,b2,r2]])
G20=matrix([[s2,x2,y2,b2,1]])

[Back to ToC](#ToC)

<a id="LieGroup"></a>
### Lie group definition

We look for the SSR group law in term of the canonical coordinates.

First define two elements for helper routines: their outputs produce a valid Python code. The output is used to define the group law in a routine below. The direct definition will execute slightly faster.

In [ ]:
gs=realsymbol("G.op(0)")
gx=realsymbol("G.op(1)")
gy=realsymbol("G.op(2)")
gb=realsymbol("G.op(3)")
gr=realsymbol("G.op(4)")
Gop=matrix([[gs,gx,gy,gb,gr]])

g1s=realsymbol("G1.op(0)")
g1x=realsymbol("G1.op(1)")
g1y=realsymbol("G1.op(2)")
g1b=realsymbol("G1.op(3)")
g1r=realsymbol("G1.op(4)")

G1op=matrix([[g1s,g1x,g1y,g1b,g1r]])

We calculate the group law in the above coordinates and output in plain notations.

In [ ]:
print_plain("Product: ",  from_ssr_matrix(to_ssr_matrix(Gop) * to_ssr_matrix(G1op)))
    
Latex("Product: $%s$" % evalm(to_ssr_matrix(G) * to_ssr_matrix(G1)))

We cut & paste the above output to define the multiplication on the group in the next routine:

In [ ]:
def ssr_mul(G, G1):
    """ SSR group multiplication of two elements""" 
    if type(G) is list:
        if len(G) == 4:
            G.append(1)
        return matrix([[(2*G[1]*G1[2]*G[4]+2*G1[0]*G[4]**2-G[3]*G1[2]**2+2*G[4]**2*G[0])/2*G[4]**(-2),\
                        G[1]-G[3]*G1[2]*G[4]**(-1)+G1[1]*G[4],\
                        G[2]+G1[2]*G[4]**(-1),\
                        G1[3]*G[4]**2+G[3],\
                        G1[4]*G[4]]])
    return matrix([[-G.op(4)**(-2)*(G.op(3)*G1.op(2)**2-2*G.op(0)*G.op(4)**2-2*G1.op(2)*G.op(4)*G.op(1)-2*G.op(4)**2*G1.op(0))/2,\
                        G1.op(1)*G.op(4)-G.op(3)*G1.op(2)*G.op(4)**(-1)+G.op(1),\
                        G1.op(2)*G.op(4)**(-1)+G.op(2),\
                        G.op(4)*(G.op(4)*G1.op(3)*G1.op(4)**(-1)+G.op(3)*G.op(4)**(-1)*G1.op(4)**(-1))*G1.op(4),G.op(4)*G1.op(4)]]).normal()    

New formulae are used to print reduced multiplication law.

In [ ]:
display(bool_highlighted(f'Restriction to shear (sub)group: ${ssr_mul(G,G1).subs({r : 1, r1 : 1})}$'))
bool_highlighted(f'Group multiplication: ${ssr_mul(G,G1)}$')

*Exercise*. We check if the above definition is accurate and agrees with the original formula.

In [ ]:
display(bool_highlighted("Multiplication the same as from matrices: $%s$" % evalm(from_ssr_matrix(to_ssr_matrix(G) * to_ssr_matrix(G1))-ssr_mul(G,G1)).normal().is_zero_matrix()))
bool_highlighted("Check associativity: $%s$" % evalm(ssr_mul(G,ssr_mul(G1,G2))-ssr_mul(ssr_mul(G,G1),G2)).normal().is_zero_matrix())

*Exercise*. Check the identity element in the SSR group.

In [ ]:
ssr_identity = matrix([[0, 0, 0, 0, 1]])
display(bool_highlighted(f'Identity from left: ${(ssr_mul(ssr_identity, G) - G).is_zero()}$' ))
bool_highlighted(f'Identity from right: ${(ssr_mul(G, ssr_identity) - G).is_zero()}$' )

Now we produce a string, which can be used to define the inverse element.

In [ ]:
print_plain('Group inverse: ', from_ssr_matrix(pow(to_ssr_matrix(Gop),-1)))

The last output can be used to create a direct routine SSR inverse:

In [ ]:
def ssr_inverse(G):
    """ SSR group inverse element"""
    if type(G) is list:
        if len(G) == 4:
            G.append(1)
        return matrix([[G[3]*G[2]**2/2+G[1]*G[2]-G[0],-(G[1]+G[3]*G[2])*G[4]**(-1),-G[2]*G[4],-G[3]*G[4]**(-1),G[4]**(-1)]])
    else:
        return matrix([[G.op(1)*G.op(2)-G.op(0)+G.op(3)*G.op(2)**2/2,-G.op(4)**(-1)*(G.op(1)+G.op(3)*G.op(2)),-G.op(4)*G.op(2),-G.op(3)*G.op(4)**(-2),G.op(4)**(-1)]])

Check the inverse method:

In [ ]:
display(bool_highlighted(f'Inverse in matrix terms coincides: ${evalm(from_ssr_matrix(pow(to_ssr_matrix(G),-1))-ssr_inverse(G)).is_zero()}$'))
display(bool_highlighted(f'Left inverse: ${(ssr_mul(G,ssr_inverse(G))-ssr_identity).is_zero()}$'))
bool_highlighted(f'Right inverse: ${(ssr_mul(ssr_inverse(G),G)-ssr_identity).is_zero()}$')
Latex(f'Inverse is: ${ssr_inverse(G)}$')

We calculate the left action on the group:

In [ ]:
bool_highlighted(f'${ssr_mul(ssr_inverse(G1),G2)}$')

[Back to ToC](#ToC)

<a id="subgroupsHS"></a>
## Subgroups and homogeneous spaces

We calculate the SSR actions on two homogeneous spaces $G/H$, where $H$ is either

+ the maximal abelian subgroup of the SSR group; or
+ the centre of the SSR group.

We use techniques and notations from [Ref.1](#refAlmalkiKisil19).

[Back to ToC](#ToC)

<a id="maxabelianSubgroup"></a>
### Maximal abelian subgroup

Here we define the maximal abelian subgroup and the respective maps:

+ projection $p: G \rightarrow G/H$;
+ section $s: G/H \rightarrow G$;
+ the complementary map $r: G \rightarrow H$.

In [ ]:
def ssr_r_map(G):
    """ r map for the SSR group homogeneous space and the maximal abelean subgroup"""  
    return matrix([[G.op(0), G.op(1), 0, G.op(3), G.op(4)]])

def ssr_p_map(G):
    """ p map (natural projection) for the SSR group homogeneous space and the maximal abelean subgroup"""  
    return ssr_mul(G, ssr_inverse(ssr_r_map(G)))

def ssr_p_map_param(G):
    """ p map (natural projection) in the parametrised form  for the SSR group homogeneous space and the maximal abelean subgroup"""  
    return ssr_p_map(G).op(2)

def ssr_s_map(y):
    """ s map (section) for the SSR group homogeneous space and the maximal abelean subgroup"""  
    return matrix([[0, 0, y, 0, 1]])

*Exercise*. Print the maps  and check the compatibility.

In [ ]:
display(Latex(f'Map $p({G})={ssr_p_map(G)}$'))
display(Latex(f'Map $s({y})={ssr_s_map(y)}$'))
display(Latex(f'Map $r({G})={ssr_r_map(G)}$'))
display(bool_highlighted(f'Maps agree: $y=p(s(y)): {(y - ssr_p_map_param(ssr_s_map(y))).is_zero()}$'))
bool_highlighted(f'Maps agree: $g=s(p(g))r(g): {evalm(G - ssr_mul(ssr_p_map(G),ssr_r_map(G))).is_zero()}$')

The group action $G: G/H \rightarrow G/H$ on the homogeneous space. 

In [ ]:
display(Latex("Action: $%s$" % ssr_p_map_param(ssr_mul(ssr_inverse(G0), ssr_s_map(y1)))))
Latex("Char: $%s$" % ssr_r_map(ssr_mul(ssr_inverse(G0), ssr_s_map(y1))))

Derivatives of the character of the representation.

In [ ]:
h=realsymbol("h", "\hbar")
h2=realsymbol("h2", "h_2")
def chi(G):
    """ General character of the SSR group for the induction of representations """ 
    return exp(2*Pi*I*(h2*G.op(3)+h*G.op(0)))

display(bool_highlighted("ds: $%s$" % chi(ssr_r_map(ssr_mul(ssr_inverse(G0), ssr_s_map(y1)))).diff(s).subs({s:0, x:0, y:0, b:0, r:1})))
display(bool_highlighted("dx: $%s$" % chi(ssr_r_map(ssr_mul(ssr_inverse(G0), ssr_s_map(y1)))).diff(x).subs({s:0, x:0, y:0, b:0, r:1})))
display(bool_highlighted("dy: $%s$" % chi(ssr_r_map(ssr_mul(ssr_inverse(G0), ssr_s_map(y1)))).diff(y).subs({s:0, x:0, y:0, b:0, r:1})))
bool_highlighted("db: $%s$" % chi(ssr_r_map(ssr_mul(ssr_inverse(G0), ssr_s_map(y1)))).diff(b).subs({s:0, x:0, y:0, b:0, r:1}))

[Back to ToC](#ToC)

<a id="centreSSRGroup"></a>
### The centre of the SSR group

Now we repeat the same steps to define homogeneous space for the centre of the SSR group.

In [ ]:
def ssr_r0_map(G):
    """ r map for the SSR group homogeneous space and its center """  
    return matrix([[G.op(0),0, 0, 0, 1]])

def ssr_p0_map(G):
    """ p map (natural projection) for the SSR group homogeneous space and its center """  
    return ssr_mul(G, ssr_inverse(ssr_r0_map(G)))

def ssr_p0_map_param(G):
    """ p map (natural projection) in parametrised form for the SSR group homogeneous space and its center """  
    Y = ssr_p0_map(G)
    return matrix([[Y.op(1), Y.op(2), Y.op(3), Y.op(4)]])

def ssr_s0_map(*args):
    """ s map (section) for the SSR group homogeneous space and its center """  
    if len(args)==1:
        if type(args[0]) is list:
            return matrix([[0, args[0][0], args[0][1], args[0][2], args[0][3]]])
        elif type(args[0]) is cginac.matrix:
            return matrix([[0, args[0].op(0), args[0].op(1), args[0].op(2), args[0].op(3)]])
    elif len(args)==3:
        return matrix([[0, args[0], args[1], args[2], 1]])
    elif len(args)==4:
        return matrix([[0, args[0], args[1], args[2], args[3]]])
    else:
        print("Do not know how to handle this number of parameters")

*Exercise*. Print the maps  and check the compatibility.

In [ ]:
display(Latex(f'p-map: $p({G})={ssr_p0_map(G)}$'))
display(Latex(f'r-map: $r({G})={ssr_r0_map(G)}$'))
display(Latex(f's-map: $s({x},{y},{b},{r})={ssr_s0_map([x,y,b,r])}$'))
display(bool_highlighted(f's-map is p-map inverse: $x=p(s(x)): {evalm(matrix([[x,y,b,r]])-ssr_p0_map_param(ssr_s0_map([x,y,b,r]))).is_zero()}$'))
bool_highlighted(f'Maps agree: ${bool(evalm(G - ssr_mul(ssr_p0_map(G), ssr_r0_map(G)))==0)}$')

The action of the SSR group on the $\mathbb{R}^4$ from the centre.

In [ ]:
display(bool_highlighted("half-Action: $%s$" % ssr_p0_map_param(ssr_mul(G0, ssr_s0_map(x1,y1,b1,1)))))
display(bool_highlighted("Action: $%s$" % ssr_p0_map_param(ssr_mul(ssr_inverse(G0), ssr_s0_map(x1,y1,b1,1)))))
bool_highlighted("Char: $%s$" % ssr_r0_map(ssr_mul(ssr_inverse(G0), ssr_s0_map(x1,y1,b1,1))))

[Back to ToC](#ToC)

<a id="indeycedRepresentations"></a>
## Induced representations of the SSR group

For the above two subgroups (the centre and the maximal abelian) we write down the induced representations.

<a id="definitionFunctions"></a>
### Definition of the symbolic functions

First we need define generic functions `f1`,...,`f6` of several variables to test properties of representations.

In [ ]:
f1_serial = function.register_new("f1", 1, "f_1")

def f1(param):
    """ A function of one variable """
    return function(f1_serial, param)

f1p_serial = function.register_new("f1p", 1, "f_{1p}")

def f1p(param):
    """ Another function of one variable """
    return function(f1p_serial, param)

f2_serial = function.register_new("f2", 2, "f_2")

def f2(param1, param2):
    """ A function of two variables """
    return function(f2_serial, param1, param2)

f2p_serial = function.register_new("f2p", 2, "f_{2p}")

def f2p(param1, param2):
    """ Another function of two variables """
    return function(f2p_serial, param1, param2)

f3_serial = function.register_new("f3", 3, "f_3")

def f3(param1, param2, param3):
    """ A function of three variables """
    return function(f3_serial, param1, param2, param3)

f4_serial = function.register_new("f4", 4, "f_4")

def f4(*args):
    """ A function of four variables which can be either provided individually 
    or as an enumerable object"""
    if len(args)==1:
        M = args[0]
        return function(f4_serial, M.op(0), M.op(1), M.op(2), M.op(3))
    elif len(args)==4:
        return function(f4_serial, args[0], args[1], args[2], args[3])
    else:
        print("Wrong number of argumentsfor f4")

f5_serial = function.register_new("f5", 5, "f_5")

def f5(param1, param2, param3, param4, param5):
    """ A function of tive variables """
    return function(f5_serial, param1, param2, param3, param4, param5)

f6_serial = function.register_new("f6", 6, "f_6")

def f6(param1, param2, param3, param4, param5, param6):
    """ A function of six variables """
    return function(f6_serial, param1, param2, param3, param4, param5, param6)

[Back to ToC](#ToC)

<a id="inductionCentre"></a>
### Representations of the SSR group induced from the centre

First we define the regular vrepresentations of the SSR group.

In [ ]:
def rho_regular(f, G1, derived = Left_action, g=matrix([[s,x,y,b,r]])):
    """ 
    The regular representations on the SSR group 
    The first pareameter define the function to be acted and 
    the last argument describes coordinates on the SSR group used in the function.
    The parameter of the shift is the second argument.
    The ation is either on the left  (derived=True) or on the right (derived=False)
    """
    if derived:
        G2 = ssr_mul(ssr_inverse(G1), g)
    else:
        G2 = ssr_mul(g, G1)
    return f.subs({g[0]: G2[0], g[1] : G2[1], g[2] : G2[2], g[3] : G2[3], g[4] : G2[4]})

Based on the previous construction of homogeneous space we define the representation.

In [ ]:
def rho0(f, G1, derived = Left_action, g=matrix([[x,y,b,r]])):
    """ 
    The representation of the SSR group induced from the centre. 
    The first pareameter define the function to be acted and 
    the last argument describes coordinates on the homogeneous space used in the function.
    The parameter of the shift is the second argument.
    The ation is either on the left  (derived=True) or on the right (derived=False)
    """
    if derived:
        G2 = ssr_mul(ssr_inverse(G1), ssr_s0_map(g))
    else:
        G2 = ssr_mul(ssr_s0_map(g), G1)

    return exp(-2*Pi*I*h*G2.op(0)) \
    * f.subs({g[0] : G2[1], g[1] : G2[2], g[2] : G2[3], g[3] : G2[4]})

Print the expression of the representation.

In [ ]:
display(Latex("The representation: $%s$" % rho0(f4(x1, y1, b1, r1), G, Left_action, matrix([[x1, y1, b1, r1]]))))
Latex("The right action: $%s$" % rho0(f4(x, y, b, r), G1, Right_action))

*Exercise*. Check the associativity (the group homomorphism) of the right action.

In [ ]:
Answer = (rho0(rho0(f4(x, y, b, r), G1, Right_action), G2, Right_action)/rho0(f4(x, y, b, r), ssr_mul(G2,G1), Right_action))
Answer = apply_laws(Answer)
bool_highlighted(f'Group homomorphism/associativity of the right action: ${(Answer-1).is_zero()}$')

*Exercise*. Check the associativity (the group homomorphism) the representation.

In [ ]:
Answer = (rho0(rho0(f4(x, y, b, r), G1), G2)/rho0(f4(x, y, b, r), ssr_mul(G2,G1)))
Answer = apply_laws(Answer)
bool_highlighted(f'Group homomorphism/associativity of the representation: ${(Answer-1).is_zero()}$')

[Back to ToC](#ToC)

<a id="infinitesimalCentre"></a>
#### Infinitesimal (derived) representations of the SSR group induced from the centre

We define the routines for calculate infinitesimal derived representations as well

In [ ]:
def der(Rho, f, i, Derived=True, g = 0):
    """
    Derived representation/Lie derivative for a SSR group representation given as the first parameter.
    The second argument is the function to act on. 
    The third parameter is the index of the basis element in the Lie algebra of the SSR group.
    The fourth parameter tells either to use the derived representation (the left action) 
    or Lie derivative (the right action).
    The last parameter is a list of variables in the function. Its zero value is to use the defaults from the used representation.
    """
    t = realsymbol("t")
    T = matrix([[0, 0, 0, 0, 1]])
    if i<4:
        T[i] = t
    else:
        T[4] = exp(t)

    if (g == 0):
        return Rho(f, T, Derived).diff(t).subs({t : 0}).normal()
    else:
        return Rho(f, T, Derived, g).diff(t).subs({t : 0}).normal()        

The procedure to find first order differential operator of a function

In [ ]:
def first_order_vec(Rho, Func, Coef, Derived = Left_action, g = 0):
    """
    First order differential operatror which is the linear combination of the derived representations.
    The first argument is a SSR group representation to use.
    The second argument is the function to act on. 
    The third parameter is the vector of coefficients for the linear combination.
    The firth parameter says either to use the derived representation or the Lie derivative.
    The last parameter is a list of variables in the function. Its zero value is to use the defaults from the used representation.
    """
    vect = []
    if type(Coef) is list:
        vect = Coef
    else:
        for i in range(Coef.nops()):
            vect.append(Coef.op(i))
    result = 0
    for i in range(len(vect)):
        result = result + vect[i]*der(Rho, Func, i, Derived, g)
    return result

This is the procedure to check commutator relations between vector fields.

In [ ]:
def check_commutator(Rho, Func, i, j, Coef, Derived = Left_action, g = 0):
    """
    Check commutators of the Lie algebra basis elements with index i and j
    for the representation Rho. 
    A symbolic function Func of the right bumber of variables need to be supplied.
    The right value of the commutator (as a list of coefficients over the Lie algebra) is supplied in Coef parameter.
    Two last parameters has the same meaning as in the derived representation function.
    """
    Val = (der(Rho, der(Rho, Func, j, Derived, g), i, Derived, g)\
        - der(Rho, der(Rho, Func, i, Derived, g), j, Derived, g)\
        - first_order_vec(Rho, Func, Coef, Derived, g)).normal()
    return Val

Here is the table of non-zero commutators to check:

In [ ]:
Commutators = [ [1, 2, [1,0,0,0,0] ],\
                [4, 1, [0,1,0,0,0] ],\
                [4, 2, [0,0,-1,0,0] ],\
                [4, 3, [0,0,0,2,0] ],\
                [3, 1, [0,0,0,0,0] ],\
                [3, 2, [0,-1,0,0,0] ] ]

def check_all_commutator(Rho, Func, Derived = Left_action):
    """ A wrapper to check all commuytators in the SSR group. """
    title = ["Checking Derived representation", "Checking Lie derivatives"]
    for j in range(2):
        print(title[j])
        for i in range(len(Commutators)):
            display(bool_highlighted("$\qquad$ Check $[%s,%s]$ is correct: $%s$" % (fields[Commutators[i][0]], fields[Commutators[i][1]],\
                                                                              check_commutator(Rho, Func, Commutators[i][0], Commutators[i][1], Commutators[i][2], j==0).is_zero())))

Output of the derived representations and Lie derivatives:

In [ ]:
print("Derived representations:")
for i in range(5):
    display(Latex(f"$\qquad$ d[{fields[i]}] = ${der(rho0, f4(x, y, b, r), i)}$"))

print("Lie derivatives:")
for i in range(5):
    display(Latex(f"$\qquad$ L[{fields[i]}] = ${der(rho0, f4(x, y, b, r), i, Right_action)}$"))

We are checking all commutators for these vector fields:

In [ ]:
check_all_commutator(rho0, f4(x,y,b,r))

[Back to ToC](#ToC)

<a id="inducedMaximal"></a>
### The representation induced from the maximal abelian subgroup 

This representation acts on the real line:

In [ ]:
def rho(f, G1, derived = Left_action, g=u):
    """ 
    The representation of the SSR group induced from the maximal abelean subgroup. 
    The first pareameter define the function to be acted and 
    the last argument describes the variable used in the function.
    The parameter of the shift is the second argument.
    The ation is either on the left  (derived=True) or on the right (derived=False)
    """
    if derived:
        G2 = ssr_mul(ssr_inverse(G1), ssr_s_map(g))
    else:
        G2 = ssr_mul(ssr_s_map(g), G1)
 
    return exp(-2*Pi*I*h*ssr_r_map(G2).op(0)) \
    * pow(G2.op(4),  - numeric(1,2)) * f.subs({g : ssr_p_map_param(G2)})

Print the formula for the representation

In [ ]:
Latex("The representation is: $%s$" % rho(f1(y1), G, True, y1))

Check associativity (group homomorphism) for the right action, this shall fail.

In [ ]:
Answer = rho(rho(f1(u), G1, Right_action), G2, Right_action)/rho(f1(u), ssr_mul(G2,G1), Right_action)
Answer = apply_laws(Answer)
Latex(f'Right action associativity fails: ${Answer}$')

Check associativity (group homomorphism) for the representation.

In [ ]:
Answer = rho(rho(f1(u), G1, Left_action), G2, Left_action)/rho(f1(u), ssr_mul(G2,G1), Left_action)
Answer = apply_laws(Answer)
bool_highlighted(f'Left action associativity holds: ${(Answer-1).is_zero()}$')


In [ ]:
print("Derived representation")
for i in range(5):
    display(bool_highlighted(f"$\qquad$ d[{fields[i]}] = ${der(rho, f1(u), i)}$"))

print("Lie deribvatioves")
for i in range(5):
    display(bool_highlighted(f"$\qquad$ L[{fields[i]}] = ${der(rho, f1(u), i, Right_action)}$"))    

We check commutators of those field, for the Lie derivative errors can
be ignored.

In [ ]:
check_all_commutator(rho, f1(u))

[Back to ToC](#ToC)

<a id="coherentStates"></a>
## Coherent states

We are studying the suitable coherent state transforms for the above representation. 

<a id="identifyingVacuum"></a>
### Identifying the vacuum vectors

We find the differential equation for vacuum. To construct useful coherent states for the induced covariant transform a vacuum vector needs to satisfy a couple of conditions, see [Ref.2](#refKisil15), [Ref.3](#refKisil17):

+ be joint eigenfunction for all representation operators $\rho(h)$ with $h$ in some subgroup $H$ (the Perelomov's condition).
+ be a null solution to certain annihilation operators (analyticity condition) [Ref.2](#refKisil15).

We are searching for all vacuums satisfying to both conditions.

The following parameters will be used to simplify expressions related
to vacuum vector.

In [ ]:
Es = realsymbol("Es", "E_s")
Ex = realsymbol("Ex", "E_x")
Ey = realsymbol("Ey", "E_y")
Er = realsymbol("Er", "E_r")
Eb = realsymbol("Eb", "E_b")
sigma = realsymbol("sigma")
tau = realsymbol("tau")
E_vals = {Eb : sigma*Er, Ex : (Er*tau-sigma)/2}
E_homogeneous = {Es : 0, Ey : 1}

Here is the complex operator annihilating the vacuum vector:

In [ ]:
## A direct input
#Annihilator = ((Ex*der(rho, f1(u), 1) + I*der(rho, f1(u), 2) + Er*der(rho, f1(u), 4) + I*Eb*der(rho, f1(u), 3))).normal().collect([f1(u),diff(f1(u),u)])
## Using the routines
Operator = matrix([[I*Es, Ex, I*Ey, I*Eb, Er]])
Annihilator = first_order_vec(rho, f1(u), Operator).normal().collect([f1(u),diff(f1(u),u)])
display(bool_highlighted(f'Annihilator of vacuum: ${Annihilator}$'))
Latex(f'Annihilator of vacuum substituted: ${Annihilator.subs(E_vals).normal().collect([f1(u),diff(f1(u),u)])}$')

We define two vacuum vectors, they are correspond to cases `Er=0` (for `V0`) and non-zero `Er` (for `V1`):

In [ ]:
a = realsymbol ("a")
V1 = exp(-Pi*I*h*(2*Ex*Er+Eb*Ey)/Er/Er*u-Pi*h*Eb/Er/2*pow(u,2)) \
    * pow(Er*u-I*Ey, -numeric(1,2)+2*Pi*h*Es/Er+Pi*h*Ey/pow(Er,3)*(2*Ex*Er+Eb*Ey))
V0 = exp(-Pi*h*I*Eb/3*pow(u,3)/Ey + Pi*h*Ex*pow(u,2)/Ey + 2*Pi*I*h*Es*u/Ey)

display(Latex(f'Vacuum for $E_r\\neq 0$: ${V1}$'))
display(Latex(f'Vacuum for $E_r\\neq 0$ with substitution: ${V1.subs(E_vals).normal()}$'))
Latex(f'Vacuum for $E_r = 0$: ${V0}$')

Check that they are solutions of the derived representations:

In [ ]:
bool_highlighted(f'V0 is annihilated by the Operator: ${first_order_vec(rho, V0, Operator.subs({Er : 0})).normal().is_zero()}$')

In [ ]:
bool_highlighted(f'V1 is annihilated by the Operator: ${first_order_vec(rho, V1, Operator).normal().is_zero()}$')

[Back to ToC](#ToC)

<a id="transmutationErNonZero"></a>
### Case of non-zero derivative in R direction (`Er ≠ 0`)

We can consider two different cases for the above two different vacuum vectors depending on either `Er` is zero or not..

We calculate the coherent state through the group action on the vacuums:

In [ ]:
display(Latex("Coherent states for $E_r\\neq 0$: $%s$" % \
                 apply_laws(rho(V1, G, Left_action).subs(E_vals).subs(E_homogeneous))))
display(Latex("Coherent states for $E_r=0$: $%s$" % \
         apply_laws(rho(V0, G, Left_action).subs({Er : 0}).subs(E_homogeneous))))
Latex("Coherent states for $E_r=0$ with substitution quadratic: $%s$" % \
         apply_laws(rho(V0, G, Left_action).subs(E_homogeneous).subs({Er : 0, Eb : 0})))

Show that for certain combinations of coefficients from `V0` and `V1` we get the same Gaussian.

In [ ]:
E_first = {Es : 0, Ex : -1, Ey : 1, Eb : 0, Er : 0}
E_second = {Es: 1/h/Pi/4, Ex : 0, Ey : 0, Eb : 2, Er : 1}
if ((V0.subs(E_first) - V1.subs(E_second)).normal().is_zero()):
    display(Latex("Both cases $E_r=0$ and $E_r\\neq 0$ with some substitution produce the Gaussian: $%s$" % \
          V0.subs(E_first)))

It shows that without a loss of generality we can put $\sigma = 1$ and
$\tau = 0$, or $E_b = E_r$ and $E_x = -1/2$.

In [ ]:
E_narrow = {Es : 0, Ex : -numeric(1,2), Ey : 1, Eb : Er}
E_strict = {Es : 0, Ex : -numeric(1,2), Ey : 1, Eb : 1, Er : 1}
Kernel = apply_laws(rho(V1.subs(E_strict), matrix([[s,x,y,b,r]]), Left_action))
Latex("The kernel: $%s$" % Kernel)

This is the function which need to be integrated in $u$ to obtain the
reproducing kernel of the image space.

In [ ]:
Repro_pre = apply_laws(V1.subs(E_strict)*conjugate(Kernel))
Latex("The pre-reproducing kernel: $%s$" % Repro_pre)

The image space shall be annihilated by the operator:

In [ ]:
Image_annihilator = first_order_vec(rho0, f4(x, y, b, r), conjugate(Operator)).normal() \
    .collect([f4(x, y, b, r), diff(f4(x, y, b, r),x), diff(f4(x, y, b, r),y), diff(f4(x, y, b, r),b), diff(f4(x, y, b, r),r)])
display(Latex(f'Annihilator of the image space: ${Image_annihilator}$'))
Latex(f'Annihilator of the image space substituted: ${Image_annihilator.subs(E_vals).normal().collect([f4(x, y, b, r), diff(f4(x, y, b, r),x), diff(f4(x, y, b, r),y), diff(f4(x, y, b, r),b), diff(f4(x, y, b, r),r)])}$')

The restricted value is:

In [ ]:
display(Latex(f'Annihilator of the image space narrow: ${Image_annihilator.subs(E_narrow).normal().collect([f4(x, y, b, r), diff(f4(x, y, b, r),x), diff(f4(x, y, b, r),y), diff(f4(x, y, b, r),b), diff(f4(x, y, b, r),r)])}$'))
Latex(f'Annihilator of the image space strict: ${Image_annihilator.subs(E_strict).normal().collect([f4(x, y, b, r), diff(f4(x, y, b, r),x), diff(f4(x, y, b, r),y), diff(f4(x, y, b, r),b), diff(f4(x, y, b, r),r)])}$')

Check if the kernel is annihilated by operator:

In [ ]:
MyKernel = Kernel.normal()
Answer = (first_order_vec(rho_regular, MyKernel, Operator.subs(E_strict), Right_action)/MyKernel*sqrt(r)).expand().normal()
display(bool_highlighted(f'Kernel is analytic: ${Answer.is_zero()}$'))

[Back to ToC](#ToC)

<a id="transmutationErZero"></a>
### The case of vanishing derivative in R direction (`Er = 0`)

For our purposes it is enough to consider the simplest possible case.

In [ ]:
Kernel = conjugate(rho(V0.subs(E_first), matrix([[0,x,y,b,r]]), True))\
         .subs(exponent_laws).normal()
Latex("The kernel: $%s$" % Kernel)

Manually inserted kernel, shall be the same as calculated

In [ ]:
MyKernel = sqrt(r)*exp(-Pi*h*((r**2-I*b)*pow(u-y,2)+2*I*(u-y)*x))
bool_highlighted(f'The manual kernel is correct: ${(apply_laws((Kernel/MyKernel).subs(E_first))-1).is_zero()}$')

Manually type the kernel in the reduced form and check it correctness:

In [ ]:
#Denom1 = (sqrt(r)*pow(-I+r*(u-y), -numeric(1,2))).normal()
MyKernel = (exp(Pi*h*(-(r**2+I*b)*pow(u-y,2)+2*I*(u-y)*x))*sqrt(r)).normal()
Delta = apply_laws(conjugate(MyKernel)/Kernel, with_power_laws)
MyKernel = (exp(Pi*h*(-(r**2-I*b)*pow(u-y,2)-2*I*(u-y)*x))*sqrt(r)).normal()
bool_highlighted(f'Manual kernel is correct: ${(Delta-1).is_zero()}$')

We calculate the reproducing kernel $K=\langle \phi_{g_1},
\phi_{g_2}\rangle = \langle \phi_{0}, \phi_{g_1^{-1}g_2}\rangle$. It
is the shift on the group of the image of the vacuum $e^{-\pi h u^2}$
, which is the function $\sqrt{\frac{r}{b+i(r^2+1)}} \exp(-\pi h y^2)
\exp(-\pi h (x-iy)/(r^2+1-ib))$. 

In [ ]:
vacuum =  sqrt(r1/(b1+I*(r1**2+1))) * exp(-Pi * h * y1**2) * exp(-Pi * h * pow(x1-I*y1,2)/(r1**2+1-I*b1))
#display(str(f'{apply_laws(rho0(vacuum, (matrix([[0,x,y,b,r]])), Left_action, matrix([[x1,y1,b1,r1]])).normal()).normal()}'))
Latex("The kernel: $%s$" % rho0(vacuum, (matrix([[0,x,y,b,r]])), Left_action, matrix([[x1,y1,b1,r1]])).normal())

[Back to ToC](#ToC)

<a id="CRannihilator"></a>
### Annihilating operators of Cauchy–Riemann type (first order)

There is two pairs of complex coordinates which define a sort of analyticity. We call the respective operators of Cauchy–Riemann type.

<a id="CRphaseSpace"></a>
#### The phase-space pair

We check that it satisfies the analyticity condition for the variable $z=x + (b+i r^2)y$..

In [ ]:
Answer = (first_order_vec(rho0, Kernel, conjugate(Operator.subs(E_first)), Right_action)/MyKernel*sqrt(r)).expand().normal()
bool_highlighted(f'Kernel satisfies the first analyticity condition: ${Answer.is_zero()}$')
#Latex(f'Kernel satisfies the first analyticity condition: {Answer}')

The form of the annihilated operator:

In [ ]:
def Cauchy_first(Func):
    """ First Cauchy-Riemann type operator on the image space """
    return -first_order_vec(rho0, Func, conjugate(Operator.subs(E_first)), Right_action)

Image_annihilator = Cauchy_first(f4(x, y, b, r)).normal().expand() \
    .collect([f4(x, y, b, r), diff(f4(x, y, b, r),x), diff(f4(x, y, b, r),y), diff(f4(x, y, b, r),b), diff(f4(x, y, b, r),r)])
bool_highlighted(f'Annihilator of the image space strict: ${Image_annihilator}$')

For the better performance some basic expressions need to be stored in
the normalised form.

In [ ]:
zn = (x+(b+I*r**2)*y).normal()
wn = (b+I*r**2).normal()

The generic solution of this equation:

In [ ]:
Generic_factor = (sqrt(r)*exp(-Pi*I*h*x*x/wn)).normal()
Generic_first = Generic_factor*f2(zn, wn)
Image_annihilator = Cauchy_first(Generic_first).normal()
bool_highlighted(f'Annihilator of the image space phase-space pair: ${Image_annihilator.is_zero()}$')

[Back to ToC](#ToC)

<a id="CRaffineGroup"></a>
#### Affine group analyticity

The second pair of variables connected by the first-order relations:

In [ ]:
Answer = (first_order_vec(rho, V0.subs(E_first), Operator.subs(E_second))).expand().normal()
bool_highlighted(f'Vacuum is annihilated by the second operator: ${Answer.is_zero()}$')

In [ ]:
def Cauchy_second(Func):
    """ The second Cauchy-Riemann type operator on the image space """
    return first_order_vec(rho0, Func, conjugate(matrix([[1/h/Pi/4,0,0,2,-I]])), Right_action)

Answer = Cauchy_second(MyKernel).expand().normal()
bool_highlighted(f'Kernel satisfies the second analyticity condition: ${Answer.is_zero()}$')

This is the respective differential operator:

In [ ]:
Answer = Cauchy_second(f4(x,y,b,r)).expand().normal()
Latex(f'Annihilator of the image space strict: ${Answer}$')

Check the generic solution of both analyticity conditions:

In [ ]:
Image_annihilator = Cauchy_second(Generic_first).expand().normal()
bool_highlighted(f'Check the second analyticity condition: ${Image_annihilator.is_zero()}$')

[Back to ToC](#ToC)

<a id="complexVariables"></a>
#### Complex symbolic variables

The above first-order differential operators prompts introduction of pairs of complex variables, which we symbolically denote by the following symbolic variables.

In [ ]:
Z = symbol("Z", "z")
W = symbol("W", "w")
complex_vars = {Z : zn, W : wn}

Z1 = symbol("Z1", "z_1")
Z2 = symbol("Z2", "z_2")
W1 = symbol("W1", "w_1")
W2 = symbol("W2", "w_2")

z2n = (x2+(b2+I*r2**2)*y2).normal()
w2n = (b2+I*r2**2).normal()
complex_vars1 = {Z1 : zn, W1 : wn}

complex_vars2 = {Z1 : zn, W1 : wn,  Z2 : z2n, W2 : w2n}

[Back to ToC](#ToC)

<a id="secondOrderAnnihilator"></a>
### Wave/Schrödinger annihilators (second order)

Search for the second-order relations. 

<a id="secondOrderAnnihilatorB"></a>
#### Second order annihilator for $B$

First we do it for the vector field $B$. The particular property of the representation is $X^2+2SB=0$:

In [ ]:
Answer = ((der(rho, der(rho, f1(u), 1), 1) + 2*der(rho, der(rho, f1(u), 3), 0)))\
    .subs(exponent_laws).normal()
bool_highlighted(f'Representation of $X^2+2SB$ is zero: ${Answer.is_zero()}$')

Therefore we can define the respective structural condition for the image.

In [ ]:
def Structural(Func, g = matrix([[x,y,b,r]])):
    """ The structural condition on the image space """
    return -(der(rho0, der(rho0, Func, 1, Right_action, g), 1, Right_action, g) \
            + 2*der(rho0, der(rho0, Func, 3, Right_action, g), 0, Right_action, g))

Answer = (Structural(MyKernel)/Generic_factor)\
    .subs(exponent_laws).expand().normal()
bool_highlighted(f'Annihilated by the first second-order operator: ${Answer.is_zero()}$')

The form of the annihilating operator.

In [ ]:
Answer = Structural(f4(x,y,b,r)).subs(exponent_laws).normal()
Latex(f'The form of the operator: ${Answer}$')

We will often prefer to have expressions in complex variables, The
next method makes such a substitution:

In [ ]:
def to_complex(Expr):
    """ Convert coordinates on the homogeneous space to complex form"""
    return Expr.subs({x : Z -(b+I*r**2)*y}).subs({b : W -I*r**2})\
        .subs({x2 : Z2 -(b2+I*r2**2)*y2}).subs({b2 : W2 -I*r2**2}).normal()

Structural action on the generic solution is:

In [ ]:
f2_derivatives = [f2(Z,W), diff(f2(Z,W),Z), diff(f2(Z,W),W),\
        diff(diff(f2(Z,W),Z),Z), diff(diff(f2(Z,W),W),Z), diff(diff(f2(Z,W),W),W)]
Generic_first_derivatives = list(map(lambda d : d.subs(complex_vars), f2_derivatives))
        
Answer = apply_laws(Structural(Generic_first)/sqrt(r)/exp(-Pi*h*x*x/(r**2-I*b))*(r**2-I*b)**2)
Answer = to_complex(Answer).expand().collect(f2_derivatives)
Latex(f'Action on the generic solution : ${Answer}$')

We check that the above equation has the simpler form.

In [ ]:
def Structural_complex(Func):
    """ The structural condition on the image space in the complex form """
    return W*diff(diff(Func,Z),Z)\
    - 4*Pi*I*h*Z*diff(Func,Z)\
    - 4*Pi*I*h*W*diff(Func,W)\
    - 2*Pi*I*h*Func

display(Latex(f'Simplified equation: ${Structural_complex(f2(Z, W))}$'))

bool_highlighted(f'The simplified equation is correct: ${(Answer- r**2*W*Structural_complex(f2(Z, W))).normal().is_zero()}$')

We check that a change of variables produces a simpler equation:

In [ ]:
Answer = (Structural_complex(1/sqrt(W)*f2(1/W, Z/W))*sqrt(W)).expand()\
          .subs(power_laws).normal()
Latex(f'Schroodinger equation for free particle: ${Answer*W}$')

[Back to ToC](#ToC)

<a id="secondOrderAnnihilatorR"></a>
#### Second order annihilator for $R$

Now a similar relation for the vector field $R$, that is $XY+YX+2SR = 2XY +2SR - S$. It comes from the
following property of the representation.

In [ ]:
Answer = (der(rho, der(rho, f1(u), 2), 1)+der(rho, der(rho, f1(u), 1), 2) + 2*der(rho, der(rho, f1(u), 4), 0))\
    .subs(exponent_laws).normal()
bool_highlighted(f'Representation of $XY+YX+2SR$ is zero: ${Answer.is_zero()}$')

Then we can verify the respective operator on the kernel.

In [ ]:
def Structural_second(Rho, Func, Derived = Left_action):
    """ The second structural condition on the image space """
    return der(Rho, der(Rho, Func, 2, Derived), 1, Derived) + der(Rho, der(Rho, Func, 1, Derived), 2, Derived) \
            + 2 * der(Rho, der(Rho, Func, 4, Derived), 0, Derived)

Answer = (Structural_second(rho, V0.subs(E_first), Left_action)*sqrt(r)).subs(exponent_laws).subs(power_laws).normal()
bool_highlighted(f'Kernel is annihilated by $XY+YX+2SR$ is: ${Answer.is_zero()}$')

The respective operator on the image space is:

In [ ]:
Answer = apply_laws(Structural_second(rho0, MyKernel, Right_action)/Generic_factor)
bool_highlighted(f'The image space is annihilated by the second second-order relation: ${Answer.is_zero()}$')

The explicit expression of this operator:

In [ ]:
Answer = Structural_second(rho0, f4(x,y,b,r), Right_action).subs(exponent_laws).subs(power_laws).normal()
Latex(f'The form of the second second-order operator: ${Answer}$')

Its restriction to the generic solution of the first-order system

In [ ]:
Answer = Structural_second(rho0, Generic_first, Right_action) /Generic_factor*(r**2-I*b)**2
Answer = to_complex(apply_laws(Answer, True)).expand().collect(f2_derivatives)
Latex(f'Action on the generic solution : ${Answer}$')

Check that it is reduces to the same equation on the image space as
the first second-order equation. 

In [ ]:
Check = (I*Answer/r**2- 2 * wn*Structural_complex(f2(Z, W))).subs(complex_vars).normal()\
    .subs(power_laws).normal()\
    .collect(Generic_first_derivatives)
bool_highlighted(f'The equation is the same: ${Check.is_zero()}$')

[Back to ToC](#ToC)

<a id="examplePlaneWave"></a>
### An example: plane wave

We work out the example of a plane wave.

In [ ]:
k = symbol("k")

Plane_wave1d = sqrt(r)/sqrt(b+I*r**2)*exp(-2*Pi*I*k*y)*exp(-Pi*h*(x+k/h)**2/(r**2 - I*b))
display(Latex(f'Plane wave: ${Plane_wave1d}$'))
Answer = to_complex(apply_laws(Plane_wave1d/Generic_factor))
display(Latex(f'Plane wave in complex form: ${Answer}$'))

We check that the plane wave transform satisfies all the requirement
for the image space. 

In [ ]:
display(bool_highlighted(f'First analicity: ${to_complex(apply_laws(Cauchy_first(Plane_wave1d))).is_zero()}$'))
display(bool_highlighted(f'Second analicity: ${to_complex(apply_laws(Cauchy_second(Plane_wave1d)*sqrt(wn))).is_zero()}$'))
bool_highlighted(f'Structural: ${to_complex(apply_laws(Structural(Plane_wave1d))).is_zero()}$')

[Back to ToC](#ToC)

<a id="exampleWavePacket"></a>
#### An example: wave packet

We work out the example of a wave packet.

In [ ]:
sigma = possymbol("sigma")
lam = realsymbol("lambda")

packet_factor = ((b*h+I*h*r**2+I*sigma)/h).normal()

Wave_packet_1d = sqrt(r)/sqrt(packet_factor)*exp(-2*Pi*I*lam*y-Pi*sigma*y**2)\
    *exp(-Pi*h*(x+(lam-I*sigma*y)/h)**2/(r**2+sigma/h - I*b))
display(Latex(f'Wave packet: ${Wave_packet_1d}$'))
Answer = to_complex(apply_laws(Wave_packet_1d/Generic_factor))
display(Latex(f'Plane wave in complex form: ${Answer}$'))

We check that the wave packet transform satisfies all the requirement
for the image space. 

In [ ]:
display(bool_highlighted(f'First analicity: ${to_complex(apply_laws(Cauchy_first(Wave_packet_1d))).is_zero()}$'))
display(bool_highlighted(f'Second analicity: ${to_complex(apply_laws(Cauchy_second(Wave_packet_1d)*sqrt(wn))).is_zero()}$'))
bool_highlighted(f'Structural: ${to_complex(apply_laws(Structural(Wave_packet_1d)/sqrt(packet_factor), True)).is_zero()}$')

These examples will be continued in the context of the Helmholtz equation.

[Back to ToC](#ToC)

<a id="metamorphism"></a>
## Metamorphism
Metamorphism is the covariant transform for the above defined coherent states. 


<a id="metamorphismBasics"></a>
### Basic properties of the metamorphism

Calculate the image of the derivative of the delta function:

In [ ]:
Answer = der(rho0, MyKernel, 2, True).subs(exponent_laws).subs(power_laws).subs({u : 0}).normal()
Latex(f'${Answer}$')

We look for the order reduction of the derivatives using variable
coefficients indeterminate

In [ ]:
X=realsymbol("X")
Y=realsymbol("Y")
B=realsymbol("B")
R=possymbol("R")
F=realsymbol("F")

f4_derivatives = [f4(x, y, b, r), diff(f4(x, y, b, r),x), diff(f4(x, y, b, r),y), diff(f4(x, y, b, r),b), diff(f4(x, y, b, r),r),\
        diff(diff(f4(x, y, b, r),x),x), diff(diff(f4(x, y, b, r),y),x), diff(diff(f4(x, y, b, r),b),x), diff(diff(f4(x, y, b, r),r),x),\
        diff(diff(f4(x, y, b, r),y),y), diff(diff(f4(x, y, b, r),b),y), diff(diff(f4(x, y, b, r),r),y),\
        diff(diff(f4(x, y, b, r),b),b), diff(diff(f4(x, y, b, r),r),b),\
        diff(diff(f4(x, y, b, r),r),r)]
correction_val1 = {X : -I*r*b+2*I*r*b+b**2/r-r**3-b**2/r, Y : I*r,  B : 0, R : 0, F : 2*I*b+b**2/r/r-r**2}

correction_val = {Y : I*r, X : -I*r*b+2*I*r*b+b**2/r-r**3-b**2/r, B : 0, R : 0, F : 2*I*b+b**2/r/r-r**2}

Dz = Cauchy_first(f4(x, y, b, r))
Answer = (der(rho0, der(rho0, f4(x,y,b,r), 2, True), 2, True) \
          - X*Dz.diff(x)  - Y*Dz.diff(y)  - B*Dz.diff(b) - R*Dz.diff(r)\
          - F*Structural(f4(x,y,b,r)))\
          .subs(correction_val)\
          .subs(exponent_laws).normal().expand()\
	.collect(f4_derivatives)
display(Latex(f'The form of the operator: ${Answer}$'))
Latex('For the values: ' + ', '.join(map(lambda a: f'${a}={correction_val[a]}$', correction_val.keys())))

We check that our calculation was correct

In [ ]:
Answer = (der(rho0, der(rho0, f4(x,y,b,r), 2, True), 2, True) \
          + I*r*((b+I*r**2)*Dz.diff(x)  - Dz.diff(y))\
          + (b+I*r**2)**2/r**2*Structural(f4(x,y,b,r)))\
          .subs(exponent_laws).normal().expand()\
	.collect(f4_derivatives)
Latex(f'The form of the operator: ${Answer}$')

Now we look on the action of this operator on the generic solution:

In [ ]:
Dz_special = Cauchy_first(Generic_first)

Answer = apply_laws((der(rho0, der(rho0, Generic_first, 2, True), 2, True) \
          + I*r*((b+I*r**2)*Dz_special.diff(x)  - Dz_special.diff(y))\
          + (b+I*r**2)**2/r**2*Structural(Generic_first))\
            /Generic_factor)

Answer = to_complex(Answer).expand().collect(f2_derivatives)
Latex(f'The complexified form of the operator on the generic solution: ${Answer}$')

Check the simpler form of this operator:

In [ ]:
def Second_der_oper(Func, z = Z, w = W):
    return  2*z*diff(Func, z) \
        + 2*w*diff(Func, w) \
        + Func

def Second_der_trans(Func, z = Z, w = W):
    return 2*Pi*I*h*w*Second_der_oper(Func, z, w)

display(Latex('The transmuted second derivative on the generic solution: $%s$' %\
                 Second_der_trans(f2(Z,W))))
bool_highlighted(f'The simpler expression is correct: ${(Answer- Second_der_trans(f2(Z,W))).normal().is_zero()}$')

[Back to ToC](#ToC)

<a id="solvingHelmholtz"></a>
### Solving the image of the Helmholtz equation

Using the Euler operator we can reduce the equation to a simpler one
for a function `f2`of two variables only.

In [ ]:
def Helmholtz(Func, K = k**2):
    """ The helmholtz operator on function Func with the square L of the wave parameter """
    return diff(diff(Func, y), y) + diff(diff(Func, y2), y2) + K * Func

def Helmholtz_trans(Func, K = k**2):
    """ The transmuted form of the Helmholts operartor, which acts on the image space """
    return Second_der_trans(Func, Z1, W1) \
        + Second_der_trans(Func, Z2, W2) \
        + K * Func

<a id="solvingPlaneWave"></a>
#### Example: plane wave solution

We check that a product of two plane waves solves the Helmholtz equation.

In [ ]:
k1=realsymbol("k1", "k_1")
k2=realsymbol("k2", "k_2")
Plane_wave = Plane_wave1d.subs({k : k1}) * Plane_wave1d.subs({x : x2, y : y2, b : b2, r : r2, k : k2})

Answer = apply_laws(to_complex(Plane_wave1d/Generic_factor), True)
Latex(f'Plane wave expression complex: ${Answer}$')

We use the above output to enter the plane wave in the complex form:

In [ ]:
Plane_wave_complex1d = exp((-2*Pi*I*h*k*Z-Pi*I*k**2)/(h*W))*W**numeric(-1,2)

In [ ]:
Answer = apply_laws(der(rho0, der(rho0, Plane_wave1d, 2), 2)/Plane_wave1d, True)
display(Latex(f'Eigenvalue of the second derivative on the plane wave: ${Answer}$')) 

Answer = apply_laws(Second_der_trans(Plane_wave_complex1d) / Plane_wave_complex1d, True)
Latex(f'${Answer}$')

In [ ]:
Plane_wave_complex = Plane_wave_complex1d.subs({Z : Z1, W : W1, k : k1}) *Plane_wave_complex1d.subs({Z : Z2, W : W2, k : k2})

Full_Plane_wave = Generic_factor*Plane_wave_complex.subs(complex_vars2)

Answer = apply_laws(Helmholtz(Full_Plane_wave, 4*Pi**2*(k1**2+k2**2))/Full_Plane_wave, True)

display(Latex(f'Plane wave: ${Full_Plane_wave}$'))
display(bool_highlighted(f'Full Helnholtz solution: ${Answer.is_zero()}$'))

Answer = apply_laws(Helmholtz_trans(Plane_wave_complex,  4*Pi**2*(k1**2+k2**2))/Plane_wave_complex)
bool_highlighted(f'Reduced Helmholtz solution: ${Answer.is_zero()}$')

[Back to ToC](#ToC)

<a id="solvingWavePaclet"></a>
#### Example: wave packet solution

We check that a product of two plane waves solves the Helmholtz equation.

In [ ]:
Wave_packet = Wave_packet_1d.subs({k : k1}) * Wave_packet_1d.subs({x : x2, y : y2, b : b2, r : r2, k : k2})

Answer = apply_laws(to_complex(Wave_packet_1d/Generic_factor), True)
Latex(f'Plane wave expression complex: ${Answer}$')

We use the above output to enter the plane wave in the complex form:

In [ ]:
Wave_packet_complex_1d = sqrt(h/(h*W+I*sigma)) \
    * exp(-(Pi*lam**2/W+2*Pi*h*lam*Z/W-Pi*I*h*sigma*Z**2/W**2)/(sigma/W-I*h))
FAC=(h+I*sigma*W).normal()
SOL=(sqrt(h)/sqrt(FAC) \
             * exp(-(I*(Pi*lam**2-k**2/(8*Pi))*W+2*I*Pi*h*lam*Z+Pi*h*sigma*Z**2)/FAC))\
             .subs({lam : k1/(2*Pi)})
display(Latex(f'${apply_laws((SOL).diff(W)/SOL,True)}$'))
bool_highlighted(f'Complex form is correct: ${(Answer-Wave_packet_complex_1d).normal().is_zero()}$')

The next chunk needs an improvement as well.

In [ ]:
Wave_packet_complex = Wave_packet_complex_1d.subs({Z : Z1, W : W1, lam : k1/(2*Pi)}) *Wave_packet_complex_1d.subs({Z : Z2, W : W2, lam : k2/(2*Pi)})

Full_Wave_packet = Generic_factor*Wave_packet_complex.subs(complex_vars2)

Answer = apply_laws(Helmholtz(Full_Wave_packet, 4*Pi**2*(k1**2+k2**2))/Full_Wave_packet, True)

#display(Latex(f'Wave packet: ${Full_Wave_packet}$'))
#display(str(f'True Helnholtz solution: {Answer}'))
#display(str(f'True Helnholtz solution: {Answer.is_zero()}'))

Answer = apply_laws(Helmholtz_trans(Wave_packet_complex,  4*Pi**2*(k1**2+k2**2))/Wave_packet_complex)
#display(Latex(f'${Answer}$'))
#bool_highlighted(f'Reduced Helmholtz solution: ${Answer.is_zero()}$')

[Back to ToC](#ToC)

<a id="HelmholtzGeneric"></a>
## Generic solution of the Helmholtz equation

We find and check the generic solution to the Helmholtz equation obtained from the metamorphism.

<a id="CartesianRepresentation"></a>
### Cartesian coordinates representation

Here we present the generic solution expressed in Cartesian coordinates.

In [ ]:
Helmholtz_factor = exp(k**2/(8*Pi*I*h)*(W1**(-1)+W2**(-1)))/sqrt(W1)/sqrt(W2)
Solution = Helmholtz_factor*f3(W2**(-1)-W1**(-1), Z1/W1, Z2/W2)
display(Latex(f'The solution is: ${Solution}$'))
Answer = (Helmholtz_trans(Solution)).normal()
bool_highlighted(f'Solution is correct: ${Answer.is_zero()}$')

Find the form of the structural condition on this solution:

In [ ]:
Generic_factor_double = Generic_factor * Generic_factor.subs({x : x2, y : y2, b : b2, r : r2})

Full_solution = Generic_factor_double * Solution.subs(complex_vars2).normal()

We check that the solution satisfies to both analyticity conditions:

In [ ]:
Answer = apply_laws(Cauchy_first(Full_solution), True)
display(bool_highlighted(f'The first Cauchy equation: ${Answer.is_zero()}$'))

Answer = apply_laws(Cauchy_second(Full_solution), True)
Answer = to_complex(Answer*r**(-2))
bool_highlighted(f'The second Cauchy equation: ${Answer.is_zero()}$')

We find the form of the structural condition for the function `f3` in
the generic solution, it turns to be the Schrodinger equation of a
free particle.

In [ ]:
Answer = apply_laws(Structural(Full_solution) / Generic_factor_double / Helmholtz_factor.subs(complex_vars2), True)
Structural1 = to_complex(Answer*r**(-2)*wn**2).normal()
display(Latex(f'The form of the first structural condition: ${Structural1}$'))

Answer = apply_laws(Structural(Full_solution, matrix([[x2,y2,b2,r2]])) / Generic_factor_double / Helmholtz_factor.subs(complex_vars2), True)
Structural2 = to_complex(Answer*r2**(-2)*w2n**2).normal()
Latex(f'The form of the second structural condition: ${Structural2}$')

Finally we are checking the unmodified Helmholtz operator on the
generic solution:

In [ ]:
Answer = apply_laws(Helmholtz(Full_solution) / Generic_factor_double / Helmholtz_factor.subs(complex_vars2), True)
Answer = to_complex(Answer)
bool_highlighted('Satisfies the full Helmholtz (subject to the strucutural conditions): $%s$' % \
                 (Answer+Structural1+Structural2).normal().is_zero())

The derivative of the generic solution for the boundary value problems.

In [ ]:
Answer = apply_laws(der(rho0, Full_solution, 2, True) / Generic_factor_double / Helmholtz_factor.subs(complex_vars2), True)
Answer = to_complex(Answer).normal()
display(Latex(f'Derivative in Y1: ${Answer}$'))

Answer = apply_laws(der(rho0, Full_solution, 2, True, matrix([[x2,y2,b2,r2]])) / Generic_factor_double / Helmholtz_factor.subs(complex_vars2), True)
Answer = to_complex(Answer).normal()
display(Latex(f'Derivative in Y2: ${Answer}$'))

[Back to ToC](#ToC)

<a id="partialPlaneWave"></a>
### Plane wave from the partial solution

We start from a partial solutions of the structural conditions to
re-create a plane wave.

In [ ]:
Partial_Schrodinger1 = exp(((k1**2-k**2/2))/(4*Pi*I*h)*W1**(-1) - I*k1*Z1/W1)
Partial_Schrodinger2 = exp(((k2**2-k**2/2))/(4*Pi*I*h)*W2**(-1) - I*k2*Z2/W2)
Partial_plane_wave = (Helmholtz_factor*Partial_Schrodinger1 * Partial_Schrodinger2).subs({k**2 : k1**2+k2**2})

Answer = apply_laws(Partial_plane_wave/(exp((k1**2+k2**2)/(8*Pi*I*h)*(W1**(-1)+W2**(-1)))/sqrt(W1)/sqrt(W2)\
                              *exp(((k1**2-k2**2))/(8*I*h*Pi)*(W1**(-1)-W2**(-1)) - I*k1*Z1/W1 - I*k2*Z2/W2)), True)
display(bool_highlighted(f'Wave has the required form: ${(Answer-1).is_zero()}$'))

Answer = apply_laws(Helmholtz_trans(Partial_plane_wave, k1**2+k2**2)*sqrt(wn)*sqrt(w2n), True)
bool_highlighted(f'Satisfies Transmuted Helmholz: ${Answer.is_zero()}$')

We make checks of the structural conditions and full Helmholtz for the full plane wave.

In [ ]:
Partial_solution = Generic_factor_double * Partial_plane_wave.subs(complex_vars2).normal()
Answer = apply_laws(Structural(Partial_solution)/Generic_factor_double*sqrt(wn)*sqrt(w2n), True)
Answer = to_complex(Answer*r**(-2))
display(bool_highlighted(f'Wave satisfies the Structural condition in the first set of variables: ${Answer.is_zero()}$'))

Answer = apply_laws(Structural(Partial_solution, matrix([[x2,y2,b2,r2]]))/Generic_factor_double*sqrt(wn)*sqrt(w2n), True)
Answer = to_complex(Answer*r**(-2))
bool_highlighted(f'Wave satisfies the Structural condition in the second set of variables: ${Answer.is_zero()}$')

In [ ]:
Answer =apply_laws(Helmholtz(Partial_solution, k1**2+k2**2)*sqrt(wn)*sqrt(w2n)*sqrt(r)*sqrt(r2))
bool_highlighted(f'It is a full Helholtz solution: ${Answer.is_zero()}$')

[Back to ToC](#ToC)

<a id="JacobiFunctions"></a>
### Jacobi (Hankel) functions


In [ ]:
Answer = der(rho0, der(rho0, der(rho0, f4(x,y,b,r), 2), 2), 3) + der(rho0, der(rho0, f4(x,y,b,r), 2), 1)\
    + der(rho0, f4(x,y,b,r), 3) - a**2*der(rho0, f4(x,y,b,r), 0)
display(Latex(f'Jacoby unmodified: ${Answer.expand().collect(f4_derivatives)}$'))

Answer = apply_laws((der(rho0, (der(rho0, der(rho0, Generic_first, 2, True), 2, True) \
          + I*r*((b+I*r**2)*Dz_special.diff(x)  - Dz_special.diff(y))\
                     + (b+I*r**2)**2/r**2*Structural(Generic_first)\
), 3)\
                     + der(rho0, der(rho0, Generic_first, 2), 1)\
    + der(rho0, Generic_first, 3) - a**2*der(rho0, Generic_first, 0)
                     )\
            /Generic_factor)

Answer = to_complex(Answer).expand().collect(f2_derivatives)
display(Latex(f'Jacoby equation on the image space: ${Answer}$'))

[Back to ToC](#ToC)

<a id="fundamentalSolution"></a>
### Fundamental solution


In [ ]:
Fundamental = pow((W2-W1)/W1/W2, numeric(-1,2))\
    *exp(Pi*I*h*pow(Z1/W1+Z2/W2, 2)/(W1**(-1)-W2**(-1)) -  a/(8*Pi*I*h)*(W1**(-1)-W2**(-1)) + c*Z1/W1 + d*Z2/W2)
#    *exp(Pi*I*h*pow(Z1/W1+Z2/W2, 2)/(W1**(-1)-W2**(-1)) - k**2/(8*Pi*I*h)*(W1**(-1)-W2**(-1)))
Full_Fundamental = Helmholtz_factor.subs({k**2 : k1**2+k2**2}) *Fundamental

#Answer = apply_laws(Helmholtz_trans(Full_Fundamental, 0)/Full_Fundamental, True)
Answer = apply_laws(Helmholtz_trans(Full_Fundamental, k1**2+k2**2)*sqrt(wn)*sqrt(w2n), True)
bool_highlighted(f'Satisfies Transmuted Helmholz: ${Answer.is_zero()}$')

The next bit does not work well.

In [ ]:
Answer =apply_laws(Helmholtz(Partial_solution, k**2)*sqrt(wn)*sqrt(w2n)*sqrt(r)*sqrt(r2)\
                   *sqrt(1/wn -1/w2n))
Answer = to_complex(Answer*wn**11*r**(-2)*wn**(-9))

bool_highlighted(f'It is a Full Helholtz solution: {Answer.is_zero()}')
Latex(f'It is a Full Helholtz solution: ${Answer}$')

[Back to ToC](#ToC)

<a id="Helmholtz3D"></a>
## Helmholtz 3D

Coordinates in the third dimension.

In [ ]:
s3=realsymbol("s3", "s_3")
x3=realsymbol("x3", "x_3")
y3=realsymbol("y3", "y_3")
b3=realsymbol("b3", "b_3")
r3=possymbol("r3", "r_3")

g3=matrix([[s3, x3, y3, b3, r3]])

Complex variables in the third dimension.

In [ ]:
Z3 = symbol("Z3", "z_3")
W3 = symbol("W3", "w_3")

z3n = (x3+(b3+I*r3**2)*y3).normal()
w3n = (b3+I*r3**2).normal()

complex_vars3 = {Z1 : zn, W1 : wn,  Z2 : z2n, W2 : w2n,  Z3 : z3n, W3 : w3n}

def to_complex3(Expr):
    """ Conversion to complex form of the triple of coordinates on the SSR group """
    return Expr.subs({x : Z -(b+I*r**2)*y}).subs({b : W -I*r**2})\
        .subs({x2 : Z2 -(b2+I*r2**2)*y2}).subs({b2 : W2 -I*r2**2})\
        .subs({x3 : Z3 -(b3+I*r3**2)*y3}).subs({b3 : W3 -I*r3**2}).normal()

We define Helmholtz operators in three dimensions:

In [ ]:
def Helmholtz3(Func, K = k**2):
    """ Three dimensional Helmholtz operator """
    return diff(diff(Func, y), y) + diff(diff(Func, y2), y2) + diff(diff(Func, y3), y3) + K * Func

def Helmholtz3_trans(Func, K = k**2):
    """ The transmuted form of three dimensional Helmholtz operator """
    return Second_der_trans(Func, Z1, W1) \
        + Second_der_trans(Func, Z2, W2) \
        + Second_der_trans(Func, Z3, W3) \
        + K * Func

[Back to ToC](#ToC)

<a id="generalHelmholtz3D"></a>
### Generic solution of the Helmholtz equation in 3D

Check the generic solution to the Helmholtz equation 

In [ ]:
Anzatz = f6(W1, W2, W3, Z1/W1, Z2/W2, Z3/W3)

Answer = Helmholtz3_trans(Anzatz).normal()
display(Latex(f'Helmholtz3 on the anzatz: ${Answer}$'))

In [ ]:
Helmholtz3_factor = exp(k**2/(12*Pi*I*h)*(W1**(-1)+W2**(-1)+W3**(-1)))/sqrt(W1)/sqrt(W2)/sqrt(W3)
Solution = Helmholtz3_factor*f5(W1**(-1)-W3**(-1), W2**(-1)-W3**(-1), Z1/W1, Z2/W2, Z3/W3)
display(Latex(f'The solution is: ${Solution}$'))
Answer = (Helmholtz3_trans(Solution)).normal()
bool_highlighted(f'Solution is correct: ${Answer.is_zero()}$')

In [ ]:
Generic_factor_triple = Generic_factor_double * Generic_factor.subs({x : x3, y : y3, b : b3, r : r3})

Full_solution = Generic_factor_triple * Solution.subs(complex_vars3).normal()

We check that the solution satisfies to all three analyticity conditions:

In [ ]:
Answer = apply_laws(Cauchy_first(Full_solution), True)
display(bool_highlighted(f'The first Cauchy equation: ${Answer.is_zero()}$'))

Answer = apply_laws(Cauchy_second(Full_solution), True)
Answer = to_complex3(Answer*r**(-2))
bool_highlighted(f'The second Cauchy equation: ${Answer.is_zero()}$')

We find the form of the structural condition for the function `f5` in
the generic solution, it turns to be the Schrodinger equation of a
free particle.

In [ ]:
Answer = apply_laws(Structural(Full_solution) / Generic_factor_triple / Helmholtz3_factor.subs(complex_vars3), True)
Structural1 = to_complex3(Answer*r**(-2)*wn**2).normal()
display(Latex(f'The form of the first structural condition: ${Structural1}$'))

Answer = apply_laws(Structural(Full_solution, matrix([[x2,y2,b2,r2]])) / Generic_factor_triple / Helmholtz3_factor.subs(complex_vars3), True)
Structural2 = to_complex3(Answer*r2**(-2)*w2n**2).normal()
display(Latex(f'The form of the second structural condition: ${Structural2}$'))

Answer = apply_laws(Structural(Full_solution, matrix([[x3,y3,b3,r3]])) / Generic_factor_triple / Helmholtz3_factor.subs(complex_vars3), True)
Structural3 = to_complex3(Answer*r3**(-2)*w3n**2).normal()
Latex(f' Form of the second structural condition in third dimension: ${Structural3}$')

[Back to ToC](#ToC)

<a id="references"></a>
### References

   <a id="refAlmalkiKisil19"></a>
1.  F. Almalki and V. V. Kisil, *Geometric dynamics of a harmonic oscillator, arbitrary minimal uncertainty states and the smallest step 3 nilpotent Lie group*, J. Phys. A: Math. Theor **52** (2019), 025301. [arXiv:1805.01399](https://arxiv.org/abs/1805.01399).

   <a id="refKisil15"></a>
2. V. V. Kisil. Uncertainty and analyticity. In V. V. Mityushev and M. V. Ruzhansky (eds.) Current trends in analysis and its applications, pages 583–590, Springer International Publishing, 2015. E-print: [arXiv:1312.4583](https://arxiv.org/abs/1312.4583).

   <a id="refKisil17"></a>
3. V. V. Kisil. Symmetry, geometry and quantization with hypercomplex numbers. In I.M.Mladenov, G. Meng, and A. Yoshioka (eds.) Geometry, integrability and quantization XVIII, pages 11–76, Bulgar. Acad. Sci., Sofia, 2017. E-print: [arXiv:1611.05650](https://arxiv.org/abs/1611.05650). 

   <a id="refKisil21c"></a>
4. V. V. Kisil, *Metamorphism---an Integral Transform Reducing the Order of a Differential Equation*, 2021, [arXiv:2105.12079](http://arxiv.org/abs/2105.12079).

   <a id="refKisil21b"></a>
5. V. V. Kisil, *Symbolic calculation for covariant transform on the SSR group*, [CodeOcean capsule](https://codeocean.com/capsule/2676832/tree/), 2021.

   <a id="EichlerZagier85"></a>
6. M. Eichler, D. Zagier, *The theory of Jacobi forms*, Birkhauser, Boston, 1985.

   <a id="refAlqurashiKisil23a"></a>
7.  T. Alqurashi and V. V. Kisil, *Metamorphism as a covariant transform for the SSR group*, 2023, [arXiv:2301.05879](http://arxiv.org/abs/2301.05879).



[Back to ToC](#ToC)